# Skeleton of Assignment 4:
    test if the distribution of 
    
    1) trip duration of bikers that ride during the day vs night
    
    2) age of bikers for trips originating in Manhattan and in Brooklyn
    
    are different. Use 3 tests: KS, Pearson's, Spearman's. 
    
    Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 
    
    For the KS do the test with the entire dataset and with a subset 200 times smaller
    
    Choose a single significant threshold for the whole exercise. 
    
    For each test phrase the Null Hypothesis in words.
    
    Describe the return of the scipy function you use in each case.
    
    State the result in terms of rejection of the Null.

In [46]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV

%pylab inline

import os
#this makes my plots pretty! but it is totally not mandatory to do it
import json
#s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
#pl.rcParams.update(s)



Populating the interactive namespace from numpy and matplotlib


# Read in data
You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [48]:
datestring = '201501'
getCitiBikeCSV(datestring)

('Downloading', '201501')
file in place, you can continue


In [50]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [ ]:
# df is the dataframe where the content of the csv file is stored
df['date'] = pd.to_datetime(df['starttime'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 
df.head()

In [ ]:
df.columns

# 0. Sample
# SPLIT BY CATEGORY

as an example I am splitting data by gender and looking at age or riders:

**H0: there is no statistical difference in the age distribution of male and female riders**
$$ \alpha = 0.05 $$

extracting the age happens in the cell below

In [ ]:
#df is the dataframe where the content of the csv file is stored
df['ageM'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['ageF'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]

In [ ]:
#dropping some data I no longer need
#... your code here...
df_KS = df.drop([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year',  u'date'],axis=1)
df_KS.head()

In [ ]:
#your df should look something like this
# look at these data carefully... you may see someinteresting values!
df_KS.describe()

In [ ]:
# dropping NaN values
df_KS['ageM'].dropna(inplace= True)
df_KS['ageF'].dropna(inplace= True)

lets split age in 10 year bins. the dataset is very large, so i could be split in smaller bins, but I will chose 10 years in the interest of time. 
the bin size choice should be a balance between properly sample the age space, have enough counts in each bin that the statistical noise is not significant (remember that is > sqrt(N)!) and the computational requirement to computatinal facilities ratio. 

the next several steps are needed if you want to code up the KS test from scratch. that is for extra credit, so if you do not want to do it you may not need to plot split the distribution in bins and create the cumulative HOWEVER it is a great idea to do it anyways to explore your data viaually! remember Ascombe's quartet!!

In [ ]:
# plotting histogramswith pandas is a bitdifferent than with pylab
# this is a VERY USEFUL syntaxfor you to knonw!
bins = np.arange(10, 99, 5)
axM = df_KS.ageM.groupby(pd.cut(df_KS.ageM, bins)).agg([count_nonzero]).plot(kind='bar', 
                                                                legend=False)
axM.set_title("male riders")
axF = df_KS.ageF.groupby(pd.cut(df_KS.ageF, bins)).agg([count_nonzero]).plot(kind='bar',
                                                                legend=False)
axF.set_title("female riders");


**Figure 1: histogrammed distribution of riders' ages by gender **
here is where you should have a nice caption that describes what I am looking at, why I am looking at it, and what I should notice

this is how the cumulative distributions look like.  Notice that i am normalizing them! if i want to reat an observed distribution like a probablility distribution i have to normalize it

In [ ]:
#print df.ageS, df.ageS.cumsum()

csM=df_KS.ageM.groupby(pd.cut(df_KS.ageM, bins)).agg([count_nonzero]).cumsum()

csF=df_KS.ageF.groupby(pd.cut(df_KS.ageF, bins)).agg([count_nonzero]).cumsum()

print (np.abs(csM / csM.max()-csF / csF.max()))

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.plot(bins[:-1] + 5, np.sqrt(csF / csF.max() - csM / csM.max())**2, 'k-',
        label = "difference")
pl.xlabel("Age")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

** Figure 2: the cumulative distribution of CitiBike riders' ages by gender** ... [a good caption here]

They look similar! But the difference gets to 10%. If I wanted to code the KS test by hand I woud have everything I need: the normalized cumulative distributions can be subtracted from each other and the max distance can calculated. 

Notice that there may be NaN values you are gonna have to deal with! 
You can do that for example with a Boolean statementsuch as  df.ageF[~np.isnan(df.ageF)] or you can use numpy functions that deal with Nan values: nansum, nanmean, nanstd...

lets run the scipy KS test

In [ ]:
import scipy.stats
#remember that your imports should all be at the top. I leave it here to hightlight that this package is needed at this point of the workflow

# 0.1 KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

the KS test in scipy returns the p-value BUT make sure you understand what the NULL is! read the documentation carefully! what is the null hypothesis that you can/cannot reject?

In [ ]:
ks = scipy.stats.ks_2samp(df_KS.ageM, df_KS.ageF)
print (ks)

**  FILL IN THE CELL BELOW!**

##### your words here!...
##### this cell is for you to tell me what the scipy KS test returned and what it means in terms of NULL HYPOTHESIS
##### to do that refer to the scipy documentation to understand the output of the scipy.stats.ks_2samp function
### KS statistic = 0.06744
### p-value = 4.0408611999082294e-168
### So reject the null hypothesis since the p-value is less than critical value 0.05.




The scipy.stats KS test already tells me the significance and the p-value. 

The next few cells are here just to show you how you would obtain the same result by hand, but they are **not required**. 

Remember: the Null hypothesis is rejected if 

$D_KS(n1,n2) > c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$

(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!! 

http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/

But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.

In [ ]:
#here is the critical values tablel. Have you chosen your significance level yet?? you should do it first thing!
from IPython.display import Image
Image("https://raw.githubusercontent.com/fedhere/PUI2017_fb55/master/plotsforclasses/ks2sample_table.png")

## 0.2 Redo the KS test with reducted dataset (a subset 200 times smaller)

In [ ]:
## your words here!...
## this cell is for you to redo the test with reducted dataset 
## and tell me what the scipy ks test returned and what it means in terms of NULL HYPOTHESIS


df_KS_sub = df_KS.sample(n=int(len(df_KS)/200),random_state=500)

len(df_KS_sub)

In [ ]:
bins = np.arange(10, 99, 5)
axM = df_KS_sub.ageM.groupby(pd.cut(df_KS_sub.ageM, bins)).agg([count_nonzero]).plot(kind='bar', 
                                                                legend=False)
axM.set_title("male riders")
axF = df_KS_sub.ageF.groupby(pd.cut(df_KS_sub.ageF, bins)).agg([count_nonzero]).plot(kind='bar',
                                                                legend=False)
axF.set_title("female riders");

** Figure 3: histogrammed distribution of riders' ages by gender**

In [ ]:
#print df.ageS, df.ageS.cumsum()

csM=df_KS_sub.ageM.groupby(pd.cut(df_KS_sub.ageM, bins)).agg([count_nonzero]).cumsum()

csF=df_KS_sub.ageF.groupby(pd.cut(df_KS_sub.ageF, bins)).agg([count_nonzero]).cumsum() 

print (np.abs(csM / csM.max()-csF / csF.max()))

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.plot(bins[:-1] + 5, np.sqrt(csF / csF.max() - csM / csM.max())**2, 'k-',
        label = "difference")
pl.xlabel("Age")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

# Now retest using a test for correlation. 

That will answer a slightly different question though - formulate the NULL appropriately. The tests for correlations (generally) requires the variable to be paired, so that I can tell if x changes does y change similarly. But the datasets are of different size! You will need to reduce them to the same size. You can do that by subsampling of the data: take only 1 ride every of 200, which you can achieve "slicing and broadcasting" the array or using one of the python function (built in python numpy.random.choice() functions for example: Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

        .. versionadded:: 1.7.0

Parameters
...

But make sure you understand how to use it! there is an option "replace" which you should think about.

In [ ]:
df_sub_m = np.random.choice(df_KS['ageM'].dropna(),size=df_KS['ageF'].count())
df_sub_f = df_KS['ageF'].dropna()

df_sub = pd.DataFrame([df_sub_m,df_sub_f]).T
df_sub.columns = ['Male','Female']

df_sub['Male'] = df_sub['Male'].sort_values().values
df_sub['Female'] = df_sub['Female'].sort_values().values

df_sub.head()

# 0.3 Pearson's  test for correlation

** notice that the Pearson's is a pairwise test: the samples need to be **
 a. the same size
 b. sorted! (how??)
    
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr



In [ ]:
# your code here!
# wrangle the data as needed
# please perform the Pearson's test 
# and tell me what you find in terms of NULL hypothesis

pr = scipy.stats.pearsonr(df_sub.Male,df_sub.Female)
print(pr)

#### Discussion:
**Pearson’s correlation coefficient = 0.99511**  
**p-value = 0.0**  
**so reject the null hypothesis because the p value is below 0.05**

# 0.3 Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

In [ ]:
# your code here!
# wrangle the data as needed
# please perform the Spearman's test and tell me what you find in terms of NULL hypothesis
spr = scipy.stats.spearmanr(df_sub.Male,df_sub.Female)
print(spr)

#### Discussion:
**Spearman’s correlation coefficient = 0.9991**  
**p-value = 0.0**  
**so reject the null hypothesis because the p value is below 0.05**

# Choose 201701 and 201707 as Sample for next two tests

In [ ]:
def download(datestring):
    #getCitiBikeCSV(datestring)
    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.csv.zip")
    ###  To move it I use the os.system() functions to run bash commands with arguments
    os.system("mv " + datestring + "-citibike-tripdata.csv.zip " + os.getenv("PUIDATA"))
    ### unzip the csv 
    os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata..csvzip")

In [ ]:
download('201701')
download('201706')

dfA = pd.read_csv(os.getenv("PUIDATA") + "/201701-citibike-tripdata.csv.zip")
dfB = pd.read_csv(os.getenv("PUIDATA") + "/201706-citibike-tripdata.csv.zip")

In [ ]:
dfA['date'] = pd.to_datetime(dfA['starttime'])
dfB['date'] = pd.to_datetime(dfB['starttime'])

In [ ]:
dfC = pd.concat([dfA,dfB],ignore_index=True)

## 1. trip duration of bikers during the day vs night

#### H0: there is no statistical difference in the trip duration distribution of bikers that ride during the day (8 am to 8 pm) vs night (8 pm to 8 am)
##### $\alpha$ = 0.05

##### Split by category
split data by day time (8 am to 8pm) and night time (8 pm to 8 am)

In [ ]:
dfC.columns

In [ ]:
dfC.groupby(df['date'].dt.hour).size()

In [ ]:
df1 = dfC

# Transform trip duration from second to minutes
df1['tripD'] = df1['tripduration'][(df1['date'].dt.hour >= 8) & (df1['date'].dt.hour < 20)] / 60
df1['tripN'] = df1['tripduration'][(df1['date'].dt.hour < 8) | (df1['date'].dt.hour >= 20)] / 60

In [ ]:
df1.head(5)

In [ ]:
df1.columns

In [ ]:
df1 = df1.drop([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender', u'date', 
       ],axis=1)
df1.head()

In [ ]:
df1.describe()

In [ ]:
df1.dropna(how='all',inplace= True)

In [ ]:
df1.describe()

In [ ]:
# plotting histogramswith pandas is a bit different than with pylab
# this is a VERY USEFUL syntaxfor you to knonw!
bins = np.arange(0,90,5)
axD = df1.tripD.groupby(pd.cut(df1.tripD, bins)).agg([count_nonzero]).plot(kind='bar', 
                                                                legend=False)
axD.set_title("trip duration during the day")
axN = df1.tripN.groupby(pd.cut(df1.tripN, bins)).agg([count_nonzero]).plot(kind='bar',
                                                                legend=False)
axN.set_title("trip duration during the night");

In [ ]:
#print df.ageS, df.ageS.cumsum()

csD=df1.tripD.groupby(pd.cut(df1.tripD, bins)).agg([count_nonzero]).cumsum()

csN=df1.tripN.groupby(pd.cut(df1.tripN, bins)).agg([count_nonzero]).cumsum() 

print (np.abs(csD / csD.max()-csN / csN.max()))

pl.plot(bins[:-1] + 5, csD / csD.max(), label = "D")
pl.plot(bins[:-1] + 5, csN / csN.max(), label = "N")
pl.plot(bins[:-1] + 5, np.sqrt(csN / csN.max() - csD / csD.max())**2, 'k-',
        label = "difference")
pl.xlabel("Trip Duration")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

### 1.1 KS test

In [ ]:
ks = scipy.stats.ks_2samp(df1.tripD, df1.tripN)
print (ks)

#### Discussion:
**statistic = 0.5238**    
**p-value = 0.0**    
**Result: reject the null hypothesis because the p-value is below critical value 0.05.** 

### 1.2 Redo the KS test with reducted dataset (a subset 200 times smaller)

In [ ]:
df1_sub = df1.sample(n=int(len(df1)/200),random_state=500)
len(df1_sub)

In [ ]:
ks_sub = scipy.stats.ks_2samp(df1_sub.tripD, df1_sub.tripN)
print (ks_sub)

### Discussion:
**KS statistic = 0.5276**    
**p-value = 0.0**         
**We can reject the null hypothesis since the p-value is below 0.05.***

### 1.3 Pearson's test

In [ ]:
pr = scipy.stats.pearsonr(df1_sub.TripDay,df1_sub.TripNight)
print(pr)

#### Discussion:
**Pearson's correlation coefficient = 0.8925**  
**the p-value = 0.0**     
**Result: The null hypothesis is rejected because the p-value is less than 0.05.**  

### 1.4 Spearman's test

In [ ]:
sr = scipy.stats.spearmanr(df1_sub.TripDay, df1_sub.TripNight)
print(sr)

#### Discussion:
**Spearman's correlation coefficient = 0.99999933283798148**  
**the p-value = 0.0**  
**Result: The null hypothesis is rejected because the p-value is less than 0.05.**  

# 2: age of bikers for trips originating in Manhattan and in Brooklyn